In [1]:
import numpy as np
import pandas as pd

In [2]:
class Comparison:
    def __init__(self, A, n):
        self.A = A
        self.n = n
        TaTb = np.array([[0.95,  1], [0.9,   1], [0.7,    1], 
                         [0.5,   1], [1,  0.95], [1,    0.9], 
                         [1,   0.7], [1,   0.5], [0.95, 0.95], 
                         [0.9, 0.9], [0.7, 0.7], [0.5,  0.5]])
        self.p_arr = np.array([0.6, 0.7, 0.8, 0.9])
        self.p_map = dict(zip(self.p_arr, [f"MSE Ratio: p = {x}" for x in self.p_arr]))
        self.template = pd.DataFrame(columns=self.p_arr)
        self.template[['T_a','T_b']] = TaTb
        self.template['Bias'] = None
    
    def theoretical(self):
        A = self.A
        n = self.n
        df = self.template.copy()
        df['Bias'] = A * (df['T_a'] + df['T_b'] - 2) + (1 - df['T_b'])
        for p in self.p_arr:
            df[p] = (1 / (16 * (p - 1/2)**2) - (A - 1/2)**2) / n / \
                    (df['Bias']**2 + ((A * df['T_a'] + (1 - A) * (1 - df['T_b'])) * (1 - A * df['T_a'] - (1 - A) * (1 - df['T_b'])) / n))
            df[p] = df[p].round(2)
        df = df.set_index(["T_a", "T_b", "Bias"]).rename(columns=self.p_map)
        return df
        
    def MCsimulation(self, size=1000, seed=123456):
        A = self.A
        n = self.n
        df = self.template.copy()
        np.random.seed(seed)
        sample = np.random.rand(size, self.n) <= A
        random_device = np.random.rand(size, n)
        mse_rd = {}
        for p in self.p_arr:
            spinner = random_device <= p
            rd_answer = sample * spinner + (1 - sample) * (1 - spinner)
            n1 = rd_answer.sum(axis=1)
            pi_hat = (p - 1) / (2 * p - 1) + n1 / n / (2 * p - 1)
            mse_rd[p] = np.sum((pi_hat - A)**2)
        for inum, irow in df.iterrows():
            truth_a = np.random.rand(size, self.n) <= irow.T_a
            truth_b = np.random.rand(size, self.n) <= irow.T_b
            trad_answer = sample * truth_a + (1 - sample) * (1 - truth_b)
            pi_trad = trad_answer.sum(axis=1) / n
            df.loc[inum, 'Bias'] = pi_trad.mean() - A
            mse_trad = np.sum((pi_trad - A)**2)
            for p in self.p_arr:
                df.loc[inum, p] = (mse_rd[p] / mse_trad).round(2)
        df = df.set_index(["T_a", "T_b", "Bias"]).rename(columns=self.p_map)
        return df

In [3]:
cp1 = Comparison(0.6, 1000)
df1_theoretical = cp1.theoretical()
df1_theoretical

MSE Ratio: p = 0.6  MSE Ratio: p = 0.7  MSE Ratio: p = 0.8  \
T_a  T_b  Bias                                                                
0.95 1.00 -0.03                5.45                1.36                0.60   
0.90 1.00 -0.06                1.62                0.40                0.18   
0.70 1.00 -0.18                0.19                0.05                0.02   
0.50 1.00 -0.30                0.07                0.02                0.01   
1.00 0.95  0.02                9.82                2.44                1.08   
     0.90  0.04                3.41                0.85                0.37   
     0.70  0.12                0.43                0.11                0.05   
     0.50  0.20                0.16                0.04                0.02   
0.95 0.95 -0.01               18.25                4.54                2.00   
0.90 0.90 -0.02                9.70                2.41                1.06   
0.70 0.70 -0.06                1.62                0.40                0.18   
0.50 0.50 -0.10                0.61                0.15                0.07   

                 MSE Ratio: p = 0.9  
T_a  T_b  Bias                       
0.95 1.00 -0.03                0.33  
0.90 1.00 -0.06                0.10  
0.70 1.00 -0.18                0.01  
0.50 1.00 -0.30                0.00  
1.00 0.95  0.02                0.60  
     0.90  0.04                0.21  
     0.70  0.12                0.03  
     0.50  0.20                0.01  
0.95 0.95 -0.01                1.11  
0.90 0.90 -0.02                0.59  
0.70 0.70 -0.06                0.10  
0.50 0.50 -0.10                0.04

In [4]:
df1_mc = cp1.MCsimulation()
df1_mc

MSE Ratio: p = 0.6 MSE Ratio: p = 0.7 MSE Ratio: p = 0.8  \
T_a  T_b  Bias                                                                 
0.95 1.00 -0.030060               5.76               1.36               0.63   
0.90 1.00 -0.060045               1.73               0.41               0.19   
0.70 1.00 -0.179530               0.21               0.05               0.02   
0.50 1.00 -0.300077               0.07               0.02               0.01   
1.00 0.95  0.019770              10.59                2.5               1.15   
     0.90  0.040050               3.63               0.86               0.39   
     0.70  0.120052               0.46               0.11               0.05   
     0.50  0.199746               0.17               0.04               0.02   
0.95 0.95 -0.010137              18.65               4.41               2.02   
0.90 0.90 -0.020103              10.48               2.48               1.14   
0.70 0.70 -0.060488               1.71                0.4               0.19   
0.50 0.50 -0.099341               0.66               0.16               0.07   

                    MSE Ratio: p = 0.9  
T_a  T_b  Bias                          
0.95 1.00 -0.030060               0.35  
0.90 1.00 -0.060045                0.1  
0.70 1.00 -0.179530               0.01  
0.50 1.00 -0.300077                0.0  
1.00 0.95  0.019770               0.64  
     0.90  0.040050               0.22  
     0.70  0.120052               0.03  
     0.50  0.199746               0.01  
0.95 0.95 -0.010137               1.12  
0.90 0.90 -0.020103               0.63  
0.70 0.70 -0.060488                0.1  
0.50 0.50 -0.099341               0.04

In [5]:
cp2 = Comparison(0.5, 1000)
df2_theoretical = cp2.theoretical()
df2_theoretical

MSE Ratio: p = 0.6  MSE Ratio: p = 0.7  MSE Ratio: p = 0.8  \
T_a  T_b  Bias                                                                 
0.95 1.00 -0.025                7.15                1.79                0.79   
0.90 1.00 -0.050                2.27                0.57                0.25   
0.70 1.00 -0.150                0.27                0.07                0.03   
0.50 1.00 -0.250                0.10                0.02                0.01   
1.00 0.95  0.025                7.15                1.79                0.79   
     0.90  0.050                2.27                0.57                0.25   
     0.70  0.150                0.27                0.07                0.03   
     0.50  0.250                0.10                0.02                0.01   
0.95 0.95  0.000               25.00                6.25                2.78   
0.90 0.90  0.000               25.00                6.25                2.78   
0.70 0.70  0.000               25.00                6.25                2.78   
0.50 0.50  0.000               25.00                6.25                2.78   

                  MSE Ratio: p = 0.9  
T_a  T_b  Bias                        
0.95 1.00 -0.025                0.45  
0.90 1.00 -0.050                0.14  
0.70 1.00 -0.150                0.02  
0.50 1.00 -0.250                0.01  
1.00 0.95  0.025                0.45  
     0.90  0.050                0.14  
     0.70  0.150                0.02  
     0.50  0.250                0.01  
0.95 0.95  0.000                1.56  
0.90 0.90  0.000                1.56  
0.70 0.70  0.000                1.56  
0.50 0.50  0.000                1.56

In [6]:
df2_mc = cp2.MCsimulation()
df2_mc

MSE Ratio: p = 0.6 MSE Ratio: p = 0.7 MSE Ratio: p = 0.8  \
T_a  T_b  Bias                                                                 
0.95 1.00 -0.025230                7.0               1.69               0.75   
0.90 1.00 -0.050279               2.23               0.54               0.24   
0.70 1.00 -0.149866               0.27               0.07               0.03   
0.50 1.00 -0.250211                0.1               0.02               0.01   
1.00 0.95  0.024410               7.38               1.78               0.79   
     0.90  0.049839               2.26               0.54               0.24   
     0.70  0.149769               0.27               0.07               0.03   
     0.50  0.249851                0.1               0.02               0.01   
0.95 0.95 -0.000260              24.29               5.86               2.59   
0.90 0.90 -0.000109              25.73                6.2               2.74   
0.70 0.70 -0.000439              25.75               6.21               2.74   
0.50 0.50  0.000768              24.91               6.01               2.65   

                    MSE Ratio: p = 0.9  
T_a  T_b  Bias                          
0.95 1.00 -0.025230               0.44  
0.90 1.00 -0.050279               0.14  
0.70 1.00 -0.149866               0.02  
0.50 1.00 -0.250211               0.01  
1.00 0.95  0.024410               0.46  
     0.90  0.049839               0.14  
     0.70  0.149769               0.02  
     0.50  0.249851               0.01  
0.95 0.95 -0.000260               1.52  
0.90 0.90 -0.000109               1.61  
0.70 0.70 -0.000439               1.61  
0.50 0.50  0.000768               1.56

In [7]:
cp3 = Comparison(0.6, 2000)
df3_theoretical = cp3.theoretical()
df3_theoretical

MSE Ratio: p = 0.6  MSE Ratio: p = 0.7  MSE Ratio: p = 0.8  \
T_a  T_b  Bias                                                                
0.95 1.00 -0.03                3.05                0.76                0.33   
0.90 1.00 -0.06                0.84                0.21                0.09   
0.70 1.00 -0.18                0.10                0.02                0.01   
0.50 1.00 -0.30                0.03                0.01                0.00   
1.00 0.95  0.02                6.03                1.50                0.66   
     0.90  0.04                1.82                0.45                0.20   
     0.70  0.12                0.22                0.05                0.02   
     0.50  0.20                0.08                0.02                0.01   
0.95 0.95 -0.01               14.12                3.51                1.55   
0.90 0.90 -0.02                5.98                1.49                0.66   
0.70 0.70 -0.06                0.84                0.21                0.09   
0.50 0.50 -0.10                0.31                0.08                0.03   

                 MSE Ratio: p = 0.9  
T_a  T_b  Bias                       
0.95 1.00 -0.03                0.19  
0.90 1.00 -0.06                0.05  
0.70 1.00 -0.18                0.01  
0.50 1.00 -0.30                0.00  
1.00 0.95  0.02                0.37  
     0.90  0.04                0.11  
     0.70  0.12                0.01  
     0.50  0.20                0.00  
0.95 0.95 -0.01                0.86  
0.90 0.90 -0.02                0.36  
0.70 0.70 -0.06                0.05  
0.50 0.50 -0.10                0.02

In [8]:
df3_mc = cp3.MCsimulation()
df3_mc

MSE Ratio: p = 0.6 MSE Ratio: p = 0.7 MSE Ratio: p = 0.8  \
T_a  T_b  Bias                                                                 
0.95 1.00 -0.030316               3.27                0.8               0.34   
0.90 1.00 -0.060352               0.91               0.22               0.09   
0.70 1.00 -0.180087               0.11               0.03               0.01   
0.50 1.00 -0.299849               0.04               0.01                0.0   
1.00 0.95  0.019734                6.7               1.64               0.69   
     0.90  0.039766               2.01               0.49               0.21   
     0.70  0.119789               0.24               0.06               0.02   
     0.50  0.200138               0.09               0.02               0.01   
0.95 0.95 -0.010475              14.78               3.61               1.53   
0.90 0.90 -0.020373               6.32               1.54               0.65   
0.70 0.70 -0.059945               0.92               0.23                0.1   
0.50 0.50 -0.100103               0.34               0.08               0.03   

                    MSE Ratio: p = 0.9  
T_a  T_b  Bias                          
0.95 1.00 -0.030316               0.19  
0.90 1.00 -0.060352               0.05  
0.70 1.00 -0.180087               0.01  
0.50 1.00 -0.299849                0.0  
1.00 0.95  0.019734               0.39  
     0.90  0.039766               0.12  
     0.70  0.119789               0.01  
     0.50  0.200138                0.0  
0.95 0.95 -0.010475               0.85  
0.90 0.90 -0.020373               0.36  
0.70 0.70 -0.059945               0.05  
0.50 0.50 -0.100103               0.02